In [14]:
import numpy as np
from pandas_datareader import get_data_fred

In [16]:

fred = ["GDPC1", "GDPDEF", "CNP16OV",
        "FEDFUNDS", 'GDPPOT', 'USRECQ']

data = (get_data_fred(fred, '1965', '2022Q4')
        .resample('Q')
        .mean()
        .to_period())
data

ConnectionError: HTTPSConnectionPool(host='fred.stlouisfed.org', port=443): Max retries exceeded with url: /graph/fredgraph.csv?id=GDPC1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000028414172790>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 232 entries, 1965Q1 to 2022Q4
Freq: Q-DEC
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GDPC1          231 non-null    float64
 1   GDPDEF         231 non-null    float64
 2   CNP16OV        232 non-null    float64
 3   FEDFUNDS       232 non-null    float64
 4   GDPPOT         232 non-null    float64
 5   USRECQ         29 non-null     float64
 6   LNSindex       232 non-null    float64
 7   output         231 non-null    float64
 8   inflation      230 non-null    float64
 9   interest rate  232 non-null    float64
 10  ygr            230 non-null    float64
 11  xgap           231 non-null    float64
dtypes: float64(12)
memory usage: 31.7 KB


In [8]:
data['CNP16OV']['1992Q3']


193024.33333333334

In [9]:
data['LNSindex'] = data['CNP16OV'] / data['CNP16OV']['1992Q3']
data['output'] = np.log(data['GDPC1'] / data['LNSindex']) * 100
data['inflation'] = np.log(data['GDPDEF'] / data['GDPDEF'].shift(1)) * 400
data['interest rate'] = data['FEDFUNDS']


data['ygr'] = data['output'].diff()
data['xgap'] = np.log(data.GDPC1/data.GDPPOT)
data['USRECQ'][data.USRECQ == 0] = np.nan

In [10]:
data

,GDPC1,GDPDEF,CNP16OV,FEDFUNDS,GDPPOT,USRECQ,LNSindex,output,inflation,interest rate,ygr,xgap
DATE,,,,,,,,,,,,
1965Q1,4064.915,17.658,125814.000000,3.976667,3993.851539,NaN,0.651804,873.815975,NaN,3.976667,NaN,0.017637
1965Q2,4116.267,17.739,126324.666667,4.080000,4036.640323,NaN,0.654449,874.666291,1.830667,4.080000,0.850317,0.019534
1965Q3,4207.782,17.808,126745.000000,4.076667,4081.098412,NaN,0.656627,876.533001,1.552876,4.076667,1.866710,0.030569
1965Q4,4304.731,17.930,127169.333333,4.166667,4126.102923,NaN,0.658825,878.476666,2.730997,4.166667,1.943664,0.042381
1966Q1,4409.518,18.046,127511.333333,4.560000,4171.864802,NaN,0.660597,880.613168,2.579506,4.560000,2.136502,0.055402
...,...,...,...,...,...,...,...,...,...,...,...,...
2021Q4,20006.181,121.708,262024.333333,0.080000,19914.290000,NaN,1.357468,959.817544,6.614350,0.080000,1.525263,0.004604
2022Q1,19924.088,124.174,263323.333333,0.120000,20005.450000,NaN,1.364198,958.911831,8.023630,0.120000,-0.905713,-0.004075
2022Q2,19895.271,126.907,263691.000000,0.770000,20101.180000,NaN,1.366102,958.627564,8.708291,0.770000,-0.284267,-0.010296


In [11]:
obs = ['ygr', 'inflation', 'interest rate']
data[obs]['1966':'2022']

,ygr,inflation,interest rate
DATE,,,
1966Q1,2.136502,2.579506,4.560000
1966Q2,0.061175,3.245140,4.913333
1966Q3,0.558296,3.851023,5.410000
1966Q4,0.518573,3.339497,5.563333
1967Q1,0.550469,1.659351,4.823333
...,...,...,...
2021Q4,1.525263,6.614350,0.080000
2022Q1,-0.905713,8.023630,0.120000
2022Q2,-0.284267,8.708291,0.770000
